In [350]:
team_dict = {
    '_id':0,
    'team_name':'',
    'type':'',
    'manager_ID':0
}
player_dict = {
    '_id':0,
    'name':'',
    'personal_info':{
        'full_name':'',
        'birthdate':'',
        'nation': [],
        'team_real':'',
        'FC_role':''
    },
    'stats':{'Pg': 0,
    'Mv': 0,
    'Mf': 0,
    'Gf': 0,
    'Gs': 0,
    'Rp': 0,
    'Rc': 0,
    'R+': 0,
    'R-': 0,
    'Ass': 0,
    'Asf': 0,
    'Amm': 0,
    'Esp': 0,
    'Au': 0,
    'Qt_A': 0,
    'Qt_I': 0}
}
transfer_dict={
    '_id':0,
    'date':0,
    'operation':'',
    'quotation_to_date':'',
    'cost':0,
    'player_ID':0,
    'team_ID_from':0,
    'team_ID_to':0
}
trophy_dict={
    '_id':0,
    'name':'',
    'season':[2020,2021],
    'prize':0,
    'manager_ID':0,
    'ranking':0
}
manager_dict={
    '_id':0,
    'name':'',
    'budget':0
}
fine_dict={
    '_id':0,
    'amount':0,
    'date':0,
    'season':[2020,2021],
    'reason':'',
    'manager_ID':0
}
loan_dict={
    '_id':0,
    'start_date':0,
    'expire_date':0,
    'redemption_right':'',
    'redemption_cost':0,
    'transfer_ID':0
}

In [365]:
from pymongo import MongoClient
import pymongo
import json
import copy
from datetime import datetime
with open('credential.json','r') as f:
    cred = json.load(f)
cluster = MongoClient(cred['cred'])
db_1 = cluster['Game']
db_2 = cluster['Game_2']
coll_pl_old = db_1['Players']
coll_pl_new = db_2['Players']
coll_man_old = db_1['Managers']
coll_man_new = db_2['Managers']
coll_fine = db_2['Fines']
coll_tr = db_2['Trophies']
coll_teams = db_2['Teams']

In [352]:
test = coll_man_old.find_one({'owner':'mario'})

In [353]:
def transform_player_dict(old_pl_dict, empty_dict=manager_dict):
    player = copy.deepcopy(empty_dict)
    old_dict = copy.deepcopy(old_pl_dict)
    
    
    player['_id'] = old_dict['_id']
    player['name'] = old_dict['name']
    player['personal_info'] = old_dict['info']['personal_info']
    player['personal_info']['birthdate'] = player['personal_info'].pop('birthdate_num')
    player['personal_info']['nation'] = player['personal_info']['nation'].split(', ')
    player['stats'] = old_dict['info']['stats']
    
    return player

In [354]:
def transform_manager_dict(old_man_dict, empty_dict=manager_dict):
    manager = copy.deepcopy(empty_dict)
    old_dict = copy.deepcopy(old_man_dict)
    
    
    manager['_id'] = old_dict['_id']
    manager['name'] = old_dict['owner']
    manager['budget'] = old_dict['budget']
    
    return manager

In [355]:
def create_trophy_dict(old_man_dict, empty_dict=trophy_dict):
    old_dict = copy.deepcopy(old_man_dict)
    
    prizes = []
    
    
    if len(old_dict['palmares']):
        
        temp0=old_dict['palmares']
        
        for prize in old_dict['palmares']:
            trophy = copy.deepcopy(empty_dict)
            
            trophy['_id'] = int(datetime.now().strftime('%Y%m%d%H%M%S'))+random.randint(0,1000)
            trophy['name'] = prize['Type']
            trophy['prize'] = prize['Prize_eur']
            trophy['season'] = [2000+int(i) for i in prize['Season'].split('/')]
            trophy['ranking'] = prize['Ranking']
            trophy['manager_ID'] = old_dict['_id']
            
            
            
            prizes = prizes +[trophy]
            
    
    
    return prizes

In [356]:
def create_fine_dict(old_man_dict, empty_dict=fine_dict):
    old_dict = copy.deepcopy(old_man_dict)
    
    fines = []
    
    if len(old_dict['fines']):
        for temp in old_dict['fines']:
            fine = copy.deepcopy(empty_dict)
            fine['_id'] = int(datetime.now().strftime('%Y%m%d%H%M%S'))+random.randint(0,1000)
            fine['reason'] = temp['Reason']
            fine['amount'] = temp['Fine_eur']
            fine['date'] = int(temp['date'].replace('/',''))
            fine['manager_ID'] = old_dict['_id']
            
            
            fines.append(fine)
    
    
    return fines

In [360]:
def create_team_dict(old_man_dict, idx, team_type, empty_dict=team_dict):
    team = copy.deepcopy(empty_dict)
    team['_id'] = idx
    team['type'] = team_type
    team['team_name'] = old_man_dict['team_name']
    team['manager_ID'] = old_man_dict['_id']
    return team